In [1]:
import ee
import time
import sys
from unidecode import unidecode
import argparse
import os.path

In [2]:
IMG_COLLECTIONS = ['MODIS/MOD09A1', 'MODIS/006/MYD11A2', 'MODIS/006/MCD12Q1']
IMG_START_DATES = ['2010-02-24', '2010-07-31', '2010-01-01']
IMG_END_DATES = ['2016-12-31', '2016-12-31', '2016-12-31']
IMG_COLLECTION_BANDS = [[0, 1, 2, 3, 4, 5, 6], [0, 4], [0]]


In [3]:
USA_SOY_FIPS_CODES = {
    "29": "MO", "20": "KS", "31": "NE", "19": "IA", "38": "ND", "46": "SD",
    "27": "MN", "05": "AR", "17": "IL", "18": "IN", "39": "OH"
}

CLEAN_NAME = lambda r, l: unidecode(r.get('properties').get(l)).lower().translate(str.maketrans('','',"'()/&-")).strip()

GET_FIPS = lambda r, l: USA_SOY_FIPS_CODES[r.get('properties').get(l)].lower()


In [4]:
def appendBand(current, previous):
    # Rename the band
    previous=ee.Image(previous)
    current = current.select(IMG_COLLECTION_BANDS[0])
    # Append it to the result (Note: only return current item on first element/iteration)
    accum = ee.Algorithms.If(ee.Algorithms.IsEqual(previous,None), current, previous.addBands(ee.Image(current)))
    # Return the accumulation
    return accum

def export_to_drive(img, fname, folder, expregion, eeuser=None, scale=500):
  expcoord = expregion.geometry().coordinates().getInfo()[0]
  
  expconfig = dict(description=fname, folder=folder, fileNamePrefix=fname, dimensions=None, region=expcoord,
                   scale=scale, crs='EPSG:4326', crsTransform=None, maxPixels=1e13)
  task = ee.batch.Export.image.toDrive(image=img.clip(expregion), **expconfig)
  task.start()
  while task.status()['state'] == 'RUNNING':
    print ("Running...")
    time.sleep(10)
  print('Done.', task.status())

In [5]:
image_collection = IMG_COLLECTIONS[0]
start_date = IMG_START_DATES[0]
end_date = IMG_END_DATES[0]
ftr_collection = 'users/nikhilarundesai/US_Counties'
boundary_filter =  [-80, 32, -104.5, 49]

In [6]:
ee.Initialize()
county_region = ee.FeatureCollection(ftr_collection)

imgcoll = ee.ImageCollection(image_collection) \
    .filterBounds(ee.Geometry.Rectangle(boundary_filter))\
    .filterDate(start_date,end_date)
img=imgcoll.iterate(appendBand)
img=ee.Image(img)

In [7]:
feature_list = county_region.toList(1e5)
feature_list_computed = feature_list.getInfo()

In [ ]:
count_already_downloaded = 0
count_filtered = 0
for idx, region in enumerate(feature_list_computed):
    if not (region.get('properties').get('STATEFP') in USA_SOY_FIPS_CODES):
        count_filtered += 1
        continue

    subunit_key = CLEAN_NAME(region, 'NAME') + "-" + GET_FIPS(region, 'STATEFP')
    file_name = 'usa' + '_' + 'sat' + '_' + subunit_key + "_" + start_date + "_" + end_date
    
    export_to_drive(img, file_name, 'satellite_dataset', ee.Feature(region), scale= 500)

print ('Ordered', len(feature_list_computed)-count_filtered, 'new tifs from GEE')
print ('Filtered', count_filtered)


